In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from collections import defaultdict
from functools import wraps

In [2]:
%matplotlib inline

In [3]:
import telepot
def send_notify(text):
    with open('../../_access_keys/telegram_token.txt', 'r') as content_file:
        token = content_file.read()
    with open('../../_access_keys/user.txt', 'r') as content_file:
        user = content_file.read()
    try:
        TelegramBot = telepot.Bot(token)
        TelegramBot.sendMessage(int(user), text)
    except:
        pass

In [4]:
TRAIN = True

In [5]:
path = 'C:/data/mlbootcamp/dataset/'
path_train = path + 'train/'
path_test = path + 'test/'
pathfeatures = 'features/'
if TRAIN:
    pathfiles = path_train
    path_subs_bs_consumption = 'subs_bs_consumption_train.csv'
    path_subs_bs_data_session = 'subs_bs_data_session_train.csv'
    path_subs_bs_voice_session = 'subs_bs_voice_session_train.csv'
    path_subs_features = 'subs_features_train.csv'
    path_subs_csi = 'subs_csi_train.csv'
else:
    pathfiles = path_test
    path_subs_bs_consumption = 'subs_bs_consumption_test.csv'
    path_subs_bs_data_session = 'subs_bs_data_session_test.csv'
    path_subs_bs_voice_session = 'subs_bs_voice_session_test.csv'
    path_subs_features = 'subs_features_test.csv'
    path_subs_csi = 'subs_csi_test.csv'

In [7]:
subs_csi = pd.read_csv(pathfiles + path_subs_csi, sep=';', decimal=',')

In [6]:
columns = ['CELL_AVAILABILITY_2G', 'CELL_AVAILABILITY_3G',
       'CELL_AVAILABILITY_4G', 'CSSR_2G', 'CSSR_3G',
       'ERAB_PS_BLOCKING_RATE_LTE', 'ERAB_PS_BLOCKING_RATE_PLMN_LTE',
       'ERAB_PS_DROP_RATE_LTE', 'HSPDSCH_CODE_UTIL_3G',
       'NODEB_CNBAP_LOAD_HARDWARE', 'PART_CQI_QPSK_LTE', 'PART_MCS_QPSK_LTE',
       'PROC_LOAD_3G', 'PSSR_2G', 'PSSR_3G', 'PSSR_LTE',
       'RAB_CS_BLOCKING_RATE_3G', 'RAB_CS_DROP_RATE_3G',
       'RAB_PS_BLOCKING_RATE_3G', 'RAB_PS_DROP_RATE_3G', 'RBU_AVAIL_DL',
       'RBU_AVAIL_DL_LTE', 'RBU_AVAIL_UL', 'RBU_OTHER_DL', 'RBU_OTHER_UL',
       'RBU_OWN_DL', 'RBU_OWN_UL', 'RRC_BLOCKING_RATE_3G',
       'RRC_BLOCKING_RATE_LTE', 'RTWP_3G', 'SHO_FACTOR', 'TBF_DROP_RATE_2G',
       'TCH_DROP_RATE_2G', 'UTIL_BRD_CPU_3G', 'UTIL_CE_DL_3G',
       'UTIL_CE_HW_DL_3G', 'UTIL_CE_UL_3G', 'UTIL_SUBUNITS_3G',
       'UL_VOLUME_LTE', 'DL_VOLUME_LTE', 'TOTAL_DL_VOLUME_3G',
       'TOTAL_UL_VOLUME_3G']

In [24]:
bs_avg_kpi = pd.read_csv(path + 'bs_avg_kpi.csv', sep=';', nrows=1000000, decimal=',')

In [8]:
bs_avg_kpi.head()

,T_DATE,CELL_LAC_ID,CELL_AVAILABILITY_2G,CELL_AVAILABILITY_3G,CELL_AVAILABILITY_4G,CSSR_2G,CSSR_3G,ERAB_PS_BLOCKING_RATE_LTE,ERAB_PS_BLOCKING_RATE_PLMN_LTE,ERAB_PS_DROP_RATE_LTE,...,TCH_DROP_RATE_2G,UTIL_BRD_CPU_3G,UTIL_CE_DL_3G,UTIL_CE_HW_DL_3G,UTIL_CE_UL_3G,UTIL_SUBUNITS_3G,UL_VOLUME_LTE,DL_VOLUME_LTE,TOTAL_DL_VOLUME_3G,TOTAL_UL_VOLUME_3G
0,1.01,275453,1,1,",02774",",93242",NaN,NaN,NaN,NaN,...,",000377",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.01,75331,1,1,",02774",",920801",NaN,NaN,NaN,NaN,...,",00062",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1.01,372021,1,1,",02774",",929026",NaN,NaN,NaN,NaN,...,",000354",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1.01,383252,1,1,",02774",",923729",NaN,NaN,NaN,NaN,...,",000587",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1.01,80247,1,1,",02774",",924741",NaN,NaN,NaN,NaN,...,",000325",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
bs_avg_kpi.describe()

,CELL_LAC_ID,CELL_AVAILABILITY_2G,CELL_AVAILABILITY_3G,CELL_AVAILABILITY_4G,CSSR_2G,CSSR_3G,ERAB_PS_BLOCKING_RATE_LTE,ERAB_PS_BLOCKING_RATE_PLMN_LTE,ERAB_PS_DROP_RATE_LTE,HSPDSCH_CODE_UTIL_3G,...,TCH_DROP_RATE_2G,UTIL_BRD_CPU_3G,UTIL_CE_DL_3G,UTIL_CE_HW_DL_3G,UTIL_CE_UL_3G,UTIL_SUBUNITS_3G,UL_VOLUME_LTE,DL_VOLUME_LTE,TOTAL_DL_VOLUME_3G,TOTAL_UL_VOLUME_3G
count,1000000.000000,553271.000000,644686.000000,243956.000000,150164.000000,482331.000000,214634.000000,108995.000000,117303.000000,0.0,...,236886.000000,0.0,245736.000000,0.0,0.0,0.0,227559.000000,227555.000000,493925.000000,482657.000000
mean,243475.795341,0.999203,0.999539,0.401048,0.928828,0.942801,0.003613,0.002415,0.015316,NaN,...,0.003145,NaN,0.013863,NaN,NaN,NaN,0.006521,0.044065,0.036261,0.010733
std,143765.150451,0.023851,0.009438,0.472866,0.010778,0.010899,0.020552,0.017054,0.014724,NaN,...,0.005780,NaN,0.013816,NaN,NaN,NaN,0.011120,0.043065,0.031208,0.010876
min,1.000000,0.000000,0.003890,0.000000,0.251601,0.000426,0.000049,0.000049,0.000000,NaN,...,0.000012,NaN,0.000002,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000
25%,119899.750000,1.000000,1.000000,0.027740,0.928827,0.943433,0.000580,0.000513,0.005846,NaN,...,0.000537,NaN,0.005287,NaN,NaN,NaN,0.001528,0.013941,0.014275,0.004547
50%,240214.000000,1.000000,1.000000,0.027740,0.931347,0.945454,0.001044,0.000854,0.011105,NaN,...,0.001320,NaN,0.009975,NaN,NaN,NaN,0.003575,0.030881,0.028059,0.008308
75%,360072.000000,1.000000,1.000000,1.000000,0.932482,0.945454,0.002294,0.001623,0.020051,NaN,...,0.003305,NaN,0.017776,NaN,NaN,NaN,0.007581,0.059952,0.048980,0.013883
max,510471.000000,1.000000,1.000000,1.000000,0.997508,0.945495,1.000000,1.000000,0.337191,NaN,...,0.228243,NaN,0.325691,NaN,NaN,NaN,0.731129,0.725428,0.707561,0.571663


In [11]:
bs_avg_kpi.columns

Index(['T_DATE', 'CELL_LAC_ID', 'CELL_AVAILABILITY_2G', 'CELL_AVAILABILITY_3G',
       'CELL_AVAILABILITY_4G', 'CSSR_2G', 'CSSR_3G',
       'ERAB_PS_BLOCKING_RATE_LTE', 'ERAB_PS_BLOCKING_RATE_PLMN_LTE',
       'ERAB_PS_DROP_RATE_LTE', 'HSPDSCH_CODE_UTIL_3G',
       'NODEB_CNBAP_LOAD_HARDWARE', 'PART_CQI_QPSK_LTE', 'PART_MCS_QPSK_LTE',
       'PROC_LOAD_3G', 'PSSR_2G', 'PSSR_3G', 'PSSR_LTE',
       'RAB_CS_BLOCKING_RATE_3G', 'RAB_CS_DROP_RATE_3G',
       'RAB_PS_BLOCKING_RATE_3G', 'RAB_PS_DROP_RATE_3G', 'RBU_AVAIL_DL',
       'RBU_AVAIL_DL_LTE', 'RBU_AVAIL_UL', 'RBU_OTHER_DL', 'RBU_OTHER_UL',
       'RBU_OWN_DL', 'RBU_OWN_UL', 'RRC_BLOCKING_RATE_3G',
       'RRC_BLOCKING_RATE_LTE', 'RTWP_3G', 'SHO_FACTOR', 'TBF_DROP_RATE_2G',
       'TCH_DROP_RATE_2G', 'UTIL_BRD_CPU_3G', 'UTIL_CE_DL_3G',
       'UTIL_CE_HW_DL_3G', 'UTIL_CE_UL_3G', 'UTIL_SUBUNITS_3G',
       'UL_VOLUME_LTE', 'DL_VOLUME_LTE', 'TOTAL_DL_VOLUME_3G',
       'TOTAL_UL_VOLUME_3G'],
      dtype='object')

In [34]:
bs_avg_kpi['CELL_AVAILABILITY_2G'].unique()

array([1.      ,      nan, 0.90935 , ..., 0.874233, 0.545238, 0.546923])

In [121]:
from collections import Counter
counter = Counter()
for i in bs_avg_kpi.values:
    _2g = i[2]
    _3g = i[3]
    _4g = i[4]
    if np.isnan(_2g) and np.isnan(_3g) and np.isnan(_4g):
        counter.update(['unknown'])
        continue
    if not np.isnan(_2g) and np.isnan(_3g) and np.isnan(_4g):
        counter.update(['2g'])
        continue
    if np.isnan(_2g) and not np.isnan(_3g) and np.isnan(_4g):
        counter.update(['3g'])
        continue
    if np.isnan(_2g) and np.isnan(_3g) and not np.isnan(_4g):
        counter.update(['4g'])
        continue
    if not np.isnan(_2g) and not np.isnan(_3g) and np.isnan(_4g):
        counter.update(['2g+3g'])
        continue        
    if np.isnan(_2g) and not np.isnan(_3g) and not np.isnan(_4g):
        counter.update(['3g+4g'])
        continue        
    #print(_2g, _3g, _4g)

In [130]:
from collections import Counter
counter = Counter()
for i in bs_avg_kpi.values:
    _2g = i[2]
    _3g = i[3]
    _4g = i[4]
    if np.isnan(_2g) and np.isnan(_3g) and np.isnan(_4g):
        counter.update(['unknown'])
        continue
        
    if not np.isnan(_2g) and np.isnan(_3g) and np.isnan(_4g):
        if _2g == 1:
            counter.update(['not 2g'])
        else:
            counter.update(['unknown-2g'])
        continue
        
    if np.isnan(_2g) and not np.isnan(_3g) and np.isnan(_4g):
        if _3g == 1:
            counter.update(['not 3g'])
        else:
            counter.update(['unknown-3g'])
        continue       
        
    if np.isnan(_2g) and np.isnan(_3g) and not np.isnan(_4g):
        if _4g == 1:
            counter.update(['not 4g'])
        else:
            counter.update(['unknown-4g'])
        continue
        
        
    if not np.isnan(_2g) and not np.isnan(_3g) and np.isnan(_4g):
        if _2g == 1 and _3g != 1:
            counter.update(['3g'])
        if _2g != 1 and _3g == 1:
            counter.update(['2g'])
        continue
        
    if np.isnan(_2g) and not np.isnan(_3g) and not np.isnan(_4g):
        counter.update(['3g+4g'])
        continue        
    #print(_2g, _3g, _4g)

In [139]:
bs_avg_kpi[columns_4g].iloc[[4]]

,CELL_AVAILABILITY_4G,DL_VOLUME_LTE,ERAB_PS_BLOCKING_RATE_LTE,ERAB_PS_BLOCKING_RATE_PLMN_LTE,ERAB_PS_DROP_RATE_LTE,PART_CQI_QPSK_LTE,PART_MCS_QPSK_LTE,PSSR_LTE,RBU_AVAIL_DL,UL_VOLUME_LTE,RBU_AVAIL_DL_LTE,RBU_AVAIL_UL,RBU_OTHER_DL,RBU_OTHER_UL,RBU_OWN_DL,RBU_OWN_UL,RRC_BLOCKING_RATE_LTE
4,0.02774,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [137]:
bs_avg_kpi.head()

,T_DATE,CELL_LAC_ID,CELL_AVAILABILITY_2G,CELL_AVAILABILITY_3G,CELL_AVAILABILITY_4G,CSSR_2G,CSSR_3G,ERAB_PS_BLOCKING_RATE_LTE,ERAB_PS_BLOCKING_RATE_PLMN_LTE,ERAB_PS_DROP_RATE_LTE,...,TCH_DROP_RATE_2G,UTIL_BRD_CPU_3G,UTIL_CE_DL_3G,UTIL_CE_HW_DL_3G,UTIL_CE_UL_3G,UTIL_SUBUNITS_3G,UL_VOLUME_LTE,DL_VOLUME_LTE,TOTAL_DL_VOLUME_3G,TOTAL_UL_VOLUME_3G
0,01.01,275453,1.0,1.0,0.02774,0.932420,NaN,NaN,NaN,NaN,...,0.000377,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,01.01,75331,1.0,1.0,0.02774,0.920801,NaN,NaN,NaN,NaN,...,0.000620,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,01.01,372021,1.0,1.0,0.02774,0.929026,NaN,NaN,NaN,NaN,...,0.000354,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,01.01,383252,1.0,1.0,0.02774,0.923729,NaN,NaN,NaN,NaN,...,0.000587,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,01.01,80247,1.0,1.0,0.02774,0.924741,NaN,NaN,NaN,NaN,...,0.000325,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [145]:
from collections import Counter
counter = Counter()
for i in enumerate(bs_avg_kpi.values):
    j = i[0]
    if j == 10:
        break
    _2g = i[1][2]
    _3g = i[1][3]
    _4g = i[1][4]
    if _2g == 1 and _3g == 1:
        counter.update(['4g'])
        print("4G", bs_avg_kpi[columns_4g].iloc[[j]].values)
        continue
    if _3g == 1 and _4g == 1:
        counter.update(['2g'])
        print("2G", bs_avg_kpi[columns_2g].iloc[[j]].values)
        continue
    if _2g == 1 and _4g == 1:
        counter.update(['3g'])
        print("3G", bs_avg_kpi[columns_3g].iloc[[j]].values)
        continue
    counter.update(['unknown'])    
    print(_2g, _3g, _4g)

4G [[0.02774     nan     nan     nan     nan     nan     nan     nan     nan
      nan     nan     nan     nan     nan     nan     nan     nan]]
4G [[0.02774     nan     nan     nan     nan     nan     nan     nan     nan
      nan     nan     nan     nan     nan     nan     nan     nan]]
4G [[0.02774     nan     nan     nan     nan     nan     nan     nan     nan
      nan     nan     nan     nan     nan     nan     nan     nan]]
4G [[0.02774     nan     nan     nan     nan     nan     nan     nan     nan
      nan     nan     nan     nan     nan     nan     nan     nan]]
4G [[0.02774     nan     nan     nan     nan     nan     nan     nan     nan
      nan     nan     nan     nan     nan     nan     nan     nan]]
4G [[0.02774     nan     nan     nan     nan     nan     nan     nan     nan
      nan     nan     nan     nan     nan     nan     nan     nan]]
4G [[0.02774     nan     nan     nan     nan     nan     nan     nan     nan
      nan     nan     nan     nan     nan     nan    

In [158]:
bs_avg_kpi[columns_2g]

,CELL_AVAILABILITY_2G,CSSR_2G,PSSR_2G,TBF_DROP_RATE_2G,TCH_DROP_RATE_2G
0,1.000000,0.932420,NaN,0.000055,0.000377
1,1.000000,0.920801,NaN,0.000059,0.000620
2,1.000000,0.929026,NaN,0.000038,0.000354
3,1.000000,0.923729,NaN,0.000077,0.000587
4,1.000000,0.924741,NaN,0.000051,0.000325
5,1.000000,0.931313,NaN,0.000053,NaN
6,1.000000,0.927652,NaN,0.000071,0.000885
7,1.000000,0.931927,NaN,0.000059,NaN
8,1.000000,0.926310,NaN,0.000035,0.000441
9,1.000000,0.926114,NaN,0.000073,0.000670


In [133]:
counter.most_common()

[('unknown', 483009), ('4g', 427006), ('2g', 89985)]

In [55]:
sum(counter.values())

907070

In [62]:
bs_avg_kpi['CELL_AVAILABILITY_4G'].min()

0.0

In [63]:
bs_avg_kpi.columns

Index(['T_DATE', 'CELL_LAC_ID', 'CELL_AVAILABILITY_2G', 'CELL_AVAILABILITY_3G',
       'CELL_AVAILABILITY_4G', 'CSSR_2G', 'CSSR_3G',
       'ERAB_PS_BLOCKING_RATE_LTE', 'ERAB_PS_BLOCKING_RATE_PLMN_LTE',
       'ERAB_PS_DROP_RATE_LTE', 'HSPDSCH_CODE_UTIL_3G',
       'NODEB_CNBAP_LOAD_HARDWARE', 'PART_CQI_QPSK_LTE', 'PART_MCS_QPSK_LTE',
       'PROC_LOAD_3G', 'PSSR_2G', 'PSSR_3G', 'PSSR_LTE',
       'RAB_CS_BLOCKING_RATE_3G', 'RAB_CS_DROP_RATE_3G',
       'RAB_PS_BLOCKING_RATE_3G', 'RAB_PS_DROP_RATE_3G', 'RBU_AVAIL_DL',
       'RBU_AVAIL_DL_LTE', 'RBU_AVAIL_UL', 'RBU_OTHER_DL', 'RBU_OTHER_UL',
       'RBU_OWN_DL', 'RBU_OWN_UL', 'RRC_BLOCKING_RATE_3G',
       'RRC_BLOCKING_RATE_LTE', 'RTWP_3G', 'SHO_FACTOR', 'TBF_DROP_RATE_2G',
       'TCH_DROP_RATE_2G', 'UTIL_BRD_CPU_3G', 'UTIL_CE_DL_3G',
       'UTIL_CE_HW_DL_3G', 'UTIL_CE_UL_3G', 'UTIL_SUBUNITS_3G',
       'UL_VOLUME_LTE', 'DL_VOLUME_LTE', 'TOTAL_DL_VOLUME_3G',
       'TOTAL_UL_VOLUME_3G'],
      dtype='object')

In [69]:
bs_avg_kpi[['PSSR_2G', 'PSSR_3G', 'PSSR_LTE']]

,PSSR_2G,PSSR_3G,PSSR_LTE
0,NaN,NaN,NaN
1,NaN,NaN,NaN
2,NaN,NaN,NaN
3,NaN,NaN,NaN
4,NaN,NaN,NaN
5,NaN,NaN,NaN
6,NaN,NaN,NaN
7,NaN,NaN,NaN
8,NaN,NaN,NaN
9,NaN,NaN,NaN


In [6]:
columns_2g = ['CELL_AVAILABILITY_2G', 'CSSR_2G', 'PSSR_2G', 'TBF_DROP_RATE_2G', 'TCH_DROP_RATE_2G']

In [7]:
columns_3g = ['CELL_AVAILABILITY_3G', 'CSSR_3G', 'HSPDSCH_CODE_UTIL_3G', 'NODEB_CNBAP_LOAD_HARDWARE', 'PROC_LOAD_3G',
             'PSSR_3G', 'RAB_CS_BLOCKING_RATE_3G', 'RAB_CS_DROP_RATE_3G', 'RAB_PS_BLOCKING_RATE_3G', 'RAB_PS_DROP_RATE_3G',
             'RRC_BLOCKING_RATE_3G', 'RTWP_3G', 'SHO_FACTOR', 'TOTAL_DL_VOLUME_3G', 'UTIL_BRD_CPU_3G', 'UTIL_CE_DL_3G',
             'UTIL_CE_HW_DL_3G', 'UTIL_CE_UL_3G', 'UTIL_SUBUNITS_3G', 'TOTAL_UL_VOLUME_3G']

In [8]:
columns_4g = ['CELL_AVAILABILITY_4G', 'DL_VOLUME_LTE', 'ERAB_PS_BLOCKING_RATE_LTE', 'ERAB_PS_BLOCKING_RATE_PLMN_LTE',
             'ERAB_PS_DROP_RATE_LTE', 'PART_CQI_QPSK_LTE', 'PART_MCS_QPSK_LTE', 'PSSR_LTE', 'RBU_AVAIL_DL', 'UL_VOLUME_LTE',
             'RBU_AVAIL_DL_LTE', 'RBU_AVAIL_UL', 'RBU_OTHER_DL', 'RBU_OTHER_UL', 'RBU_OWN_DL', 'RBU_OWN_UL', 'RRC_BLOCKING_RATE_LTE']

In [82]:
len((set(columns_4g))) == len(columns_4g)

True

In [85]:
print(bs_avg_kpi.shape)
print(bs_avg_kpi[columns_2g].dropna().shape)
print(bs_avg_kpi[columns_3g].dropna().shape)
print(bs_avg_kpi[columns_4g].dropna().shape)

(1000000, 44)
(0, 5)
(0, 20)
(0, 17)


In [86]:
bs_avg_kpi[columns_2g].head()

,CELL_AVAILABILITY_2G,CSSR_2G,PSSR_2G,TBF_DROP_RATE_2G,TCH_DROP_RATE_2G
0,1.0,0.932420,NaN,0.000055,0.000377
1,1.0,0.920801,NaN,0.000059,0.000620
2,1.0,0.929026,NaN,0.000038,0.000354
3,1.0,0.923729,NaN,0.000077,0.000587
4,1.0,0.924741,NaN,0.000051,0.000325


In [118]:
bs_avg_kpi[columns_3g].head()

,CELL_AVAILABILITY_3G,CSSR_3G,HSPDSCH_CODE_UTIL_3G,NODEB_CNBAP_LOAD_HARDWARE,PROC_LOAD_3G,PSSR_3G,RAB_CS_BLOCKING_RATE_3G,RAB_CS_DROP_RATE_3G,RAB_PS_BLOCKING_RATE_3G,RAB_PS_DROP_RATE_3G,RRC_BLOCKING_RATE_3G,RTWP_3G,SHO_FACTOR,TOTAL_DL_VOLUME_3G,UTIL_BRD_CPU_3G,UTIL_CE_DL_3G,UTIL_CE_HW_DL_3G,UTIL_CE_UL_3G,UTIL_SUBUNITS_3G,TOTAL_UL_VOLUME_3G
0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [119]:
bs_avg_kpi[columns_4g].head()

,CELL_AVAILABILITY_4G,DL_VOLUME_LTE,ERAB_PS_BLOCKING_RATE_LTE,ERAB_PS_BLOCKING_RATE_PLMN_LTE,ERAB_PS_DROP_RATE_LTE,PART_CQI_QPSK_LTE,PART_MCS_QPSK_LTE,PSSR_LTE,RBU_AVAIL_DL,UL_VOLUME_LTE,RBU_AVAIL_DL_LTE,RBU_AVAIL_UL,RBU_OTHER_DL,RBU_OTHER_UL,RBU_OWN_DL,RBU_OWN_UL,RRC_BLOCKING_RATE_LTE
0,0.02774,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.02774,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.02774,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.02774,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.02774,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
columns_avg = ['CELL_AVAILABILITY_2G', 'CELL_AVAILABILITY_3G',
       'CELL_AVAILABILITY_4G', 'CSSR_2G', 'CSSR_3G',
       'ERAB_PS_BLOCKING_RATE_LTE', 'ERAB_PS_BLOCKING_RATE_PLMN_LTE',
       'ERAB_PS_DROP_RATE_LTE', 'HSPDSCH_CODE_UTIL_3G',
       'NODEB_CNBAP_LOAD_HARDWARE', 'PART_CQI_QPSK_LTE', 'PART_MCS_QPSK_LTE',
       'PROC_LOAD_3G', 'PSSR_2G', 'PSSR_3G', 'PSSR_LTE',
       'RAB_CS_BLOCKING_RATE_3G', 'RAB_CS_DROP_RATE_3G',
       'RAB_PS_BLOCKING_RATE_3G', 'RAB_PS_DROP_RATE_3G', 'RBU_AVAIL_DL',
       'RBU_AVAIL_DL_LTE', 'RBU_AVAIL_UL', 'RBU_OTHER_DL', 'RBU_OTHER_UL',
       'RBU_OWN_DL', 'RBU_OWN_UL', 'RRC_BLOCKING_RATE_3G',
       'RRC_BLOCKING_RATE_LTE', 'RTWP_3G', 'SHO_FACTOR', 'TBF_DROP_RATE_2G',
       'TCH_DROP_RATE_2G', 'UTIL_BRD_CPU_3G', 'UTIL_CE_DL_3G',
       'UTIL_CE_HW_DL_3G', 'UTIL_CE_UL_3G', 'UTIL_SUBUNITS_3G',
       'UL_VOLUME_LTE', 'DL_VOLUME_LTE', 'TOTAL_DL_VOLUME_3G',
       'TOTAL_UL_VOLUME_3G']

In [10]:
columns_chhn = ['AVEUSERNUMBER', 'AVEUSERNUMBER_PLMN',
       'AVR_DL_HSPA_USER_3G', 'AVR_DL_R99_USER_3G', 'AVR_DL_USER_3G',
       'AVR_DL_USER_LTE', 'AVR_TX_POWER_3G', 'AVR_UL_HSPA_USER',
       'AVR_UL_R99_USER', 'AVR_UL_USER_3G', 'AVR_UL_USER_LTE',
       'DL_AVR_THROUGHPUT_3G', 'DL_AVR_THROUGHPUT_LTE',
       'DL_AVR_THROUGHPUT_R99', 'DL_MEAN_USER_THROUGHPUT_LTE',
       'DL_MEAN_USER_THROUGHPUT_DL_2G', 'DL_MEAN_USER_THROUGHPUT_HSPA3G',
       'DL_MEAN_USER_THROUGHPUT_PLTE', 'DL_MEAN_USER_THROUGHPUT_REL93G',
       'HSDPA_USERS_3G', 'HSUPA_USERS_3G', 'RBU_USED_DL', 'RBU_USED_UL',
       'RELATIVE_RBU_USED_DL', 'RELATIVE_RBU_USED_UL', 'RELATIVE_TX_POWER_3G',
       'UL_AVR_THROUGHPUT_3G', 'UL_AVR_THROUGHPUT_LTE',
       'UL_AVR_THROUGHPUT_R99', 'UL_MEAN_USER_THROUGHPUT_LTE',
       'UL_MEAN_USER_THROUGHPUT_HS3G', 'UL_MEAN_USER_THROUGHPUT_PLTE',
       'UL_MEAN_USER_THROUGHPUT_REL93G']

In [11]:
voice_columns = ['CELL_AVAILABILITY_2G', 'CELL_AVAILABILITY_3G', 'CELL_AVAILABILITY_4G', 'CSSR_2G', 'CSSR_3G',
                 'NODEB_CNBAP_LOAD_HARDWARE', 'PROC_LOAD_3G', 'RAB_CS_BLOCKING_RATE_3G', 'RAB_CS_DROP_RATE_3G',
                 'RBU_AVAIL_DL_LTE', 'RBU_AVAIL_DL', 'RBU_AVAIL_UL', 'RBU_OTHER_DL', 'RBU_OTHER_UL', 'RBU_OWN_DL', 'RBU_OWN_UL',
                 'RRC_BLOCKING_RATE_3G', 'RTWP_3G', 'SHO_FACTOR', 'TCH_DROP_RATE_2G', 'UTIL_BRD_CPU_3G', 'UTIL_CE_DL_3G',
                 'UTIL_CE_HW_DL_3G', 'UTIL_CE_UL_3G', 'UTIL_SUBUNITS_3G', 'AVR_TX_POWER_3G',
                 'RELATIVE_RBU_USED_DL', 'RELATIVE_RBU_USED_UL', 'RELATIVE_TX_POWER_3G'
                ]

In [12]:
data_columns = ['CELL_AVAILABILITY_2G', 'CELL_AVAILABILITY_3G', 'CELL_AVAILABILITY_4G', 'ERAB_PS_BLOCKING_RATE_LTE',
                'ERAB_PS_BLOCKING_RATE_PLMN_LTE', 'ERAB_PS_DROP_RATE_LTE', 'HSPDSCH_CODE_UTIL_3G', 'NODEB_CNBAP_LOAD_HARDWARE',
                'PART_CQI_QPSK_LTE', 'PART_MCS_QPSK_LTE', 'PROC_LOAD_3G', 'PSSR_2G', 'PSSR_3G', 'PSSR_LTE', 
                'RAB_PS_BLOCKING_RATE_3G', 'RAB_PS_DROP_RATE_3G', 'RBU_AVAIL_DL_LTE', 'RBU_AVAIL_DL', 'RBU_AVAIL_UL', 'RBU_OTHER_DL',
                'RBU_OTHER_UL', 'RBU_OWN_DL', 'RBU_OWN_UL', 'RRC_BLOCKING_RATE_3G', 'RRC_BLOCKING_RATE_LTE', 'RTWP_3G',
                'SHO_FACTOR', 'TBF_DROP_RATE_2G', 'UTIL_BRD_CPU_3G', 'UTIL_CE_DL_3G', 'UTIL_CE_HW_DL_3G', 'UTIL_CE_UL_3G',
                'UTIL_SUBUNITS_3G', 'UL_VOLUME_LTE', 'DL_VOLUME_LTE', 'TOTAL_DL_VOLUME_3G', 'TOTAL_UL_VOLUME_3G', 'AVEUSERNUMBER',
                'AVEUSERNUMBER_PLMN', 'AVR_DL_HSPA_USER_3G', 'AVR_DL_R99_USER_3G', 'AVR_DL_USER_3G', 'AVR_DL_USER_LTE',
                'AVR_TX_POWER_3G', 'AVR_UL_HSPA_USER', 'AVR_UL_R99_USER', 'AVR_UL_USER_3G', 'AVR_UL_USER_LTE',
                'DL_AVR_THROUGHPUT_3G', 'DL_AVR_THROUGHPUT_LTE', 'DL_AVR_THROUGHPUT_R99', 'DL_MEAN_USER_THROUGHPUT_LTE',
                'DL_MEAN_USER_THROUGHPUT_DL_2G', 'DL_MEAN_USER_THROUGHPUT_HSPA3G', 'DL_MEAN_USER_THROUGHPUT_PLTE',
                'DL_MEAN_USER_THROUGHPUT_REL93G', 'HSDPA_USERS_3G', 'HSUPA_USERS_3G', 'RBU_USED_DL', 'RBU_USED_UL',
                'RELATIVE_RBU_USED_DL', 'RELATIVE_RBU_USED_UL', 'RELATIVE_TX_POWER_3G', 'UL_AVR_THROUGHPUT_3G',
                'UL_AVR_THROUGHPUT_LTE', 'UL_AVR_THROUGHPUT_R99', 'UL_MEAN_USER_THROUGHPUT_LTE', 'UL_MEAN_USER_THROUGHPUT_HS3G',
                'UL_MEAN_USER_THROUGHPUT_PLTE', 'UL_MEAN_USER_THROUGHPUT_REL93G'
               ]

In [98]:
def internet(x):
    if x in data_columns:
        return "internet"
    return ""
def voice(x):
    if x in voice_columns:
        return "voice"
    return ""

for i in columns_chhn:
    print("{}\t{},{}".format(i, internet(i), voice(i)))

AVEUSERNUMBER	internet,
AVEUSERNUMBER_PLMN	internet,
AVR_DL_HSPA_USER_3G	internet,
AVR_DL_R99_USER_3G	internet,
AVR_DL_USER_3G	internet,
AVR_DL_USER_LTE	internet,
AVR_TX_POWER_3G	internet,voice
AVR_UL_HSPA_USER	internet,
AVR_UL_R99_USER	internet,
AVR_UL_USER_3G	internet,
AVR_UL_USER_LTE	internet,
DL_AVR_THROUGHPUT_3G	internet,
DL_AVR_THROUGHPUT_LTE	internet,
DL_AVR_THROUGHPUT_R99	internet,
DL_MEAN_USER_THROUGHPUT_LTE	internet,
DL_MEAN_USER_THROUGHPUT_DL_2G	internet,
DL_MEAN_USER_THROUGHPUT_HSPA3G	internet,
DL_MEAN_USER_THROUGHPUT_PLTE	internet,
DL_MEAN_USER_THROUGHPUT_REL93G	internet,
HSDPA_USERS_3G	internet,
HSUPA_USERS_3G	internet,
RBU_USED_DL	internet,
RBU_USED_UL	internet,
RELATIVE_RBU_USED_DL	internet,voice
RELATIVE_RBU_USED_UL	internet,voice
RELATIVE_TX_POWER_3G	internet,voice
UL_AVR_THROUGHPUT_3G	internet,
UL_AVR_THROUGHPUT_LTE	internet,
UL_AVR_THROUGHPUT_R99	internet,
UL_MEAN_USER_THROUGHPUT_LTE	internet,
UL_MEAN_USER_THROUGHPUT_HS3G	internet,
UL_MEAN_USER_THROUGHPUT_PLTE	inter

In [112]:
subs_features = pd.read_csv(pathfiles + path_subs_features, sep=';', nrows=1000000, decimal=',')

In [100]:
subs_features.head()

,SNAP_DATE,COM_CAT#1,SK_ID,COM_CAT#2,COM_CAT#3,BASE_TYPE,ACT,ARPU_GROUP,COM_CAT#7,COM_CAT#8,...,COM_CAT#25,COM_CAT#26,COM_CAT#27,COM_CAT#28,COM_CAT#29,COM_CAT#30,COM_CAT#31,COM_CAT#32,COM_CAT#33,COM_CAT#34
0,01.08.01,4,3339,80,5,2,1,12.0,1,1489.0,...,1,1,0.031696,0.076724,0.040279,0.0133,0.083382,0.027579,0.103896,2.0
1,01.08.01,4,238,75,5,2,1,7.0,1,945.0,...,1,1,0.015893,0.012981,0.000000,0.0000,0.048930,0.027579,0.103896,3.0
2,01.08.01,4,4669,43,5,2,1,3.0,3,1033.0,...,1,1,0.000415,0.000000,0.000000,0.0000,0.009397,0.027579,0.103896,3.0
3,01.08.01,4,2555,54,5,2,1,12.0,1,512.0,...,1,1,0.046510,0.039486,0.000829,0.0000,0.196676,0.027579,0.103896,1.0
4,01.08.01,4,2291,38,5,2,1,7.0,1,1268.0,...,1,1,0.005393,0.007385,0.003884,0.0000,0.047297,0.027579,0.103896,6.0


In [105]:
subs_features.columns

Index(['SNAP_DATE', 'COM_CAT#1', 'SK_ID', 'COM_CAT#2', 'COM_CAT#3',
       'BASE_TYPE', 'ACT', 'ARPU_GROUP', 'COM_CAT#7', 'COM_CAT#8',
       'DEVICE_TYPE_ID', 'INTERNET_TYPE_ID', 'REVENUE', 'ITC', 'VAS',
       'RENT_CHANNEL', 'ROAM', 'COST', 'COM_CAT#17', 'COM_CAT#18',
       'COM_CAT#19', 'COM_CAT#20', 'COM_CAT#21', 'COM_CAT#22', 'COM_CAT#23',
       'COM_CAT#24', 'COM_CAT#25', 'COM_CAT#26', 'COM_CAT#27', 'COM_CAT#28',
       'COM_CAT#29', 'COM_CAT#30', 'COM_CAT#31', 'COM_CAT#32', 'COM_CAT#33',
       'COM_CAT#34'],
      dtype='object')

In [103]:
subs_features['SNAP_DATE'].unique()

array(['01.08.01', '01.01.02', '01.05.02', '01.06.01', '01.12.01',
       '01.07.01', '01.04.02', '01.10.01', '01.03.02', '01.09.01',
       '01.11.01', '01.02.02'], dtype=object)

In [109]:
for i in subs_features.columns:
    print(i)
    print(subs_features[i].unique())

SNAP_DATE
['01.08.01' '01.01.02' '01.05.02' '01.06.01' '01.12.01' '01.07.01'
 '01.04.02' '01.10.01' '01.03.02' '01.09.01' '01.11.01' '01.02.02']
COM_CAT#1
[4 3 1 5 7 6 8 2]
SK_ID
[3339  238 4669 ...  820  350 1426]
COM_CAT#2
[80 75 43 54 38 51 36 61 59  5 47 50 70  9 57 65 45 24 16 66 53 28  6 68
 46 79  1 20 11 30 60 41  4 14 10 17 23 48 64 69 31 33 83 22 73 74 49 39
 77 63 78 26 82 55 34 71 25 35  2 21 56 67 42 40 15 12 62 29 44 13  3 52
 76 19 81 58  7 72 27 37 32 18  8]
COM_CAT#3
[ 5  1  9  4 11  2 13 15 17  6  8  3 12 16 10  7 14]
BASE_TYPE
[2 1]
ACT
[1 0]
ARPU_GROUP
[12.  7.  3.  5.  6.  4.  8. 11.  9. 10.  2.  1. nan]
COM_CAT#7
[1 3 4 2 6 7 9 8 5]
COM_CAT#8
[1489.  945. 1033. ... 1434. 1143. 1057.]
DEVICE_TYPE_ID
[ 5.  3.  7.  6. nan  8.  2.  4.  1.]
INTERNET_TYPE_ID
[ 2.  3.  1. nan]
REVENUE
[0.153024 0.121336 0.109565 ... 0.132015 0.131669 0.148063]
ITC
[0.081527 0.012981 0.       ... 0.051615 0.034802 0.070341]
VAS
[0.104168 0.084167 0.098985 ... 0.114738 0.104011 0.109271]
R

In [113]:
for i in subs_features.columns:
    print(i)
    print(subs_features[i].unique())

SNAP_DATE
['01.10.01' '01.07.01' '01.06.01' '01.01.02' '01.08.01' '01.04.02'
 '01.05.02' '01.11.01' '01.09.01' '01.02.02' '01.05.01' '01.12.01'
 '01.03.02']
COM_CAT#1
[8 4 3 1 6 5 7 2]
SK_ID
[6954 9663 7895 ... 9224  569 1677]
COM_CAT#2
[44 12 52  3 76 19 40 62 15 81 13 29 38 54 43  9 59  5 80 61 75 47 51 50
 36 70  6 16 65 45 68 11 28 53 46 57 79 66 24 20  1 30 55 71 25 35 21 56
 42 34  2 17 48 23 60 31 64  4 14 69 83 41 33 10 22 49 63 77 73 82 39 78
 26 74  8 32 27 72  7 37 58 18]
COM_CAT#3
[12  5  8  1  2  3  6  4 15 13  9 11  7 10 14 17 16]
BASE_TYPE
[1 2]
ACT
[1 0]
ARPU_GROUP
[12.  8.  7.  9.  1.  6. 11.  2.  5.  3. 10.  4. nan]
COM_CAT#7
[3 1 4 2 7 8 9 6 5]
COM_CAT#8
[  nan  952. 1322. ... 1468.  652.  361.]
DEVICE_TYPE_ID
[ 3.  5.  6.  7.  8. nan  4.  1.  2.]
INTERNET_TYPE_ID
[ 3.  2.  1. nan]
REVENUE
[0.140993 0.125821 0.123145 ... 0.17066  0.169464 0.121402]
ITC
[0.036439 0.000493 0.00119  ... 0.094548 0.019674 0.006591]
VAS
[0.101207 0.084167 0.093465 ... 0.097068 0.142555 0.

In [116]:
subs_bs_data_session = pd.read_csv(pathfiles + path_subs_bs_data_session, sep=';', decimal=',')

In [117]:
subs_bs_data_session.head()

,SK_ID,CELL_LAC_ID,DATA_VOL_MB,START_TIME
0,3853,61951,3.561184e-06,04.04 05:00:00
1,3853,61951,2.732445e-07,04.04 06:00:00
2,3853,61951,6.467041e-07,04.04 07:00:00
3,3853,61951,7.513883e-06,04.04 08:00:00
4,3853,61951,9.002802e-05,04.04 17:00:00


In [159]:
bs_avg_kpi.head()

,T_DATE,CELL_LAC_ID,CELL_AVAILABILITY_2G,CELL_AVAILABILITY_3G,CELL_AVAILABILITY_4G,CSSR_2G,CSSR_3G,ERAB_PS_BLOCKING_RATE_LTE,ERAB_PS_BLOCKING_RATE_PLMN_LTE,ERAB_PS_DROP_RATE_LTE,...,TCH_DROP_RATE_2G,UTIL_BRD_CPU_3G,UTIL_CE_DL_3G,UTIL_CE_HW_DL_3G,UTIL_CE_UL_3G,UTIL_SUBUNITS_3G,UL_VOLUME_LTE,DL_VOLUME_LTE,TOTAL_DL_VOLUME_3G,TOTAL_UL_VOLUME_3G
0,01.01,275453,1.0,1.0,0.02774,0.932420,NaN,NaN,NaN,NaN,...,0.000377,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,01.01,75331,1.0,1.0,0.02774,0.920801,NaN,NaN,NaN,NaN,...,0.000620,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,01.01,372021,1.0,1.0,0.02774,0.929026,NaN,NaN,NaN,NaN,...,0.000354,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,01.01,383252,1.0,1.0,0.02774,0.923729,NaN,NaN,NaN,NaN,...,0.000587,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,01.01,80247,1.0,1.0,0.02774,0.924741,NaN,NaN,NaN,NaN,...,0.000325,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [191]:
bs_avg_kpi.head()

,T_DATE,CELL_LAC_ID,CELL_AVAILABILITY_2G,CELL_AVAILABILITY_3G,CELL_AVAILABILITY_4G,CSSR_2G,CSSR_3G,ERAB_PS_BLOCKING_RATE_LTE,ERAB_PS_BLOCKING_RATE_PLMN_LTE,ERAB_PS_DROP_RATE_LTE,...,TCH_DROP_RATE_2G,UTIL_BRD_CPU_3G,UTIL_CE_DL_3G,UTIL_CE_HW_DL_3G,UTIL_CE_UL_3G,UTIL_SUBUNITS_3G,UL_VOLUME_LTE,DL_VOLUME_LTE,TOTAL_DL_VOLUME_3G,TOTAL_UL_VOLUME_3G
0,01.01,275453,1.0,1.0,0.02774,0.932420,NaN,NaN,NaN,NaN,...,0.000377,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,01.01,75331,1.0,1.0,0.02774,0.920801,NaN,NaN,NaN,NaN,...,0.000620,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,01.01,372021,1.0,1.0,0.02774,0.929026,NaN,NaN,NaN,NaN,...,0.000354,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,01.01,383252,1.0,1.0,0.02774,0.923729,NaN,NaN,NaN,NaN,...,0.000587,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,01.01,80247,1.0,1.0,0.02774,0.924741,NaN,NaN,NaN,NaN,...,0.000325,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [203]:
bs_avg_kpi[bs_avg_kpi['CELL_LAC_ID'] == 106103][columns_4g]

,CELL_AVAILABILITY_4G,DL_VOLUME_LTE,ERAB_PS_BLOCKING_RATE_LTE,ERAB_PS_BLOCKING_RATE_PLMN_LTE,ERAB_PS_DROP_RATE_LTE,PART_CQI_QPSK_LTE,PART_MCS_QPSK_LTE,PSSR_LTE,RBU_AVAIL_DL,UL_VOLUME_LTE,RBU_AVAIL_DL_LTE,RBU_AVAIL_UL,RBU_OTHER_DL,RBU_OTHER_UL,RBU_OWN_DL,RBU_OWN_UL,RRC_BLOCKING_RATE_LTE
428,NaN,0.112799,0.000295,0.000295,0.025896,0.139805,0.447853,0.000009,0.000002,0.007639,NaN,0.000002,NaN,NaN,0.172076,0.046229,0.000554
456718,NaN,0.267825,0.000335,0.000335,0.046055,0.081059,0.144103,0.000009,0.000002,0.016642,NaN,0.000002,NaN,NaN,0.221762,0.061228,0.000822
790335,NaN,0.121834,0.000555,0.000555,0.032472,0.177815,0.382834,0.000009,0.000002,0.014717,NaN,0.000002,NaN,NaN,0.198049,0.061659,0.000491


In [242]:
cells_2g = set()
cells_3g = set()
cells_4g = set()

In [238]:
%%time
for i in enumerate(bs_avg_kpi.values):
    j = i[0]
    if j == 400:
        break
        
    data = bs_avg_kpi[list(columns_2g)].iloc[[j]].values
    #print('2g', data)
    count_2g = (data.size - np.count_nonzero(np.isnan(data))) / data.size
    #print('2g', count_2g)
    
    data = bs_avg_kpi[list(columns_3g)].iloc[[j]].values
    #print('3g', data)
    count_3g = (data.size - np.count_nonzero(np.isnan(data))) / data.size
    #print('3g', count_3g)
    
    data = bs_avg_kpi[list(columns_4g)].iloc[[j]].values
    #print('4g', data)
    count_4g = (data.size - np.count_nonzero(np.isnan(data))) / data.size
    #print('4g', count_4g)
    
    cell_id = bs_avg_kpi[['CELL_LAC_ID']].iloc[[j]].values[0][0]
    #print(cell_id)
    if count_2g > count_3g and count_2g > count_4g:
        cells_2g.add(cell_id)
    elif count_3g > count_2g and count_3g > count_4g:
        cells_3g.add(cell_id)
    else:
        cells_4g.add(cell_id)

Wall time: 1min 8s


In [29]:
bs_avg_kpi.shape

(8941383, 44)

In [20]:
columns

Index(['T_DATE', 'CELL_LAC_ID', 'CELL_AVAILABILITY_2G', 'CELL_AVAILABILITY_3G',
       'CELL_AVAILABILITY_4G', 'CSSR_2G', 'CSSR_3G',
       'ERAB_PS_BLOCKING_RATE_LTE', 'ERAB_PS_BLOCKING_RATE_PLMN_LTE',
       'ERAB_PS_DROP_RATE_LTE', 'HSPDSCH_CODE_UTIL_3G',
       'NODEB_CNBAP_LOAD_HARDWARE', 'PART_CQI_QPSK_LTE', 'PART_MCS_QPSK_LTE',
       'PROC_LOAD_3G', 'PSSR_2G', 'PSSR_3G', 'PSSR_LTE',
       'RAB_CS_BLOCKING_RATE_3G', 'RAB_CS_DROP_RATE_3G',
       'RAB_PS_BLOCKING_RATE_3G', 'RAB_PS_DROP_RATE_3G', 'RBU_AVAIL_DL',
       'RBU_AVAIL_DL_LTE', 'RBU_AVAIL_UL', 'RBU_OTHER_DL', 'RBU_OTHER_UL',
       'RBU_OWN_DL', 'RBU_OWN_UL', 'RRC_BLOCKING_RATE_3G',
       'RRC_BLOCKING_RATE_LTE', 'RTWP_3G', 'SHO_FACTOR', 'TBF_DROP_RATE_2G',
       'TCH_DROP_RATE_2G', 'UTIL_BRD_CPU_3G', 'UTIL_CE_DL_3G',
       'UTIL_CE_HW_DL_3G', 'UTIL_CE_UL_3G', 'UTIL_SUBUNITS_3G',
       'UL_VOLUME_LTE', 'DL_VOLUME_LTE', 'TOTAL_DL_VOLUME_3G',
       'TOTAL_UL_VOLUME_3G'],
      dtype='object')

In [19]:
cells_2g = set()
cells_3g = set()
cells_4g = set()

In [28]:
bs_avg_kpi = pd.read_csv(path + 'bs_avg_kpi.csv', sep=';', skiprows=50000000, nrows=10000000, decimal=',')

In [15]:
#bs_avg_kpi = pd.read_csv(path + 'bs_avg_kpi.csv', sep=';', usecols=list(columns_2g), decimal=',')

In [21]:
%%time
for z in range(6):
    if z == 0:
        bs_avg_kpi = pd.read_csv(path + 'bs_avg_kpi.csv', sep=';', skiprows=z*10000000, nrows=10000000, decimal=',')
    else:
        bs_avg_kpi = pd.read_csv(path + 'bs_avg_kpi.csv', sep=';', skiprows=z*10000000, nrows=10000000, decimal=',', header=None)
    if z == 0:
        columns = bs_avg_kpi.columns
        columns_2g = set(columns_2g)
        columns_3g = set(columns_3g)
        columns_4g = set(columns_4g)
        size_2g = len(columns_2g)
        size_3g = len(columns_3g)
        size_4g = len(columns_4g)
    for i in enumerate(bs_avg_kpi.values):
        j = i[0]
        if j % 100000 == 0:
            print(j)
        count_2g = 0
        count_3g = 0
        count_4g = 0
        for k in enumerate(zip(columns, i[1])):
            if k[0] == 0:
                continue
            is_nan = False
            try:
                is_nan = np.isnan(k[1][1])
            except:
                print(k[1][1])
            if is_nan:
                if k[1][0] in columns_2g:
                    count_2g += 1
                elif k[1][0] in columns_3g:
                    count_3g += 1
                elif k[1][0] in columns_4g:
                    count_4g += 1

        count_2g = (size_2g - count_2g) / size_2g
        count_3g = (size_3g - count_3g) / size_3g
        count_4g = (size_4g - count_4g) / size_4g
        cell_id = i[1][1]

        if count_2g > count_3g and count_2g > count_4g:
            cells_2g.add(cell_id)
        elif count_3g > count_2g and count_3g > count_4g:
            cells_3g.add(cell_id)
        else:
            cells_4g.add(cell_id)

0
100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1600000
1700000
1800000
1900000
2000000
2100000
2200000
2300000
2400000
2500000
2600000
2700000
2800000
2900000
3000000
3100000
3200000
3300000
3400000
3500000
3600000
3700000
3800000
3900000
4000000
4100000
4200000
4300000
4400000
4500000
4600000
4700000
4800000
4900000
5000000
5100000
5200000
5300000
5400000
5500000
5600000
5700000
5800000
5900000
6000000
6100000
6200000
6300000
6400000
6500000
6600000
6700000
6800000
6900000
7000000
7100000
7200000
7300000
7400000
7500000
7600000
7700000
7800000
7900000
8000000
8100000
8200000
8300000
8400000
8500000
8600000
8700000
8800000
8900000
9000000
9100000
9200000
9300000
9400000
9500000
9600000
9700000
9800000
9900000
0
100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1600000
1700000
1800000
1900000
2000000
2100000
2200000
2300000
2400000
2500000
2600000
2700000
280000

In [22]:
send_notify('complete parse')

In [31]:
bs_avg_kpi.head()

,0,1,2,3,4,5,6,7,8,9,...,34,35,36,37,38,39,40,41,42,43
0,12.05,325057,NaN,1.000000,NaN,NaN,0.936686,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.026281,0.014998
1,12.05,31796,NaN,0.997627,NaN,NaN,0.945454,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.024260,0.006505
2,12.05,271149,NaN,0.997627,NaN,NaN,0.945454,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.018312,0.004470
3,12.05,203648,NaN,0.997800,NaN,NaN,0.877788,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.023933,0.005393
4,12.05,287835,NaN,0.997627,NaN,NaN,0.880437,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.036617,0.006759


In [25]:
len(cells_2g.intersection(cells_4g))

342

In [26]:
len(cells_2g.intersection(cells_3g))

3660

In [27]:
len(cells_3g.intersection(cells_4g))

3125

In [24]:
print('2g', len(cells_2g))
print('3g', len(cells_3g))
print('4g', len(cells_4g))

2g 120455
3g 219457
4g 115660


In [205]:
2g 130
3g 237
4g 33

600

In [30]:
pd.DataFrame(data=list(cells_4g), columns=['cell_id']).to_csv(path + '4g.csv')

In [34]:
325057 in cells_4g

False

In [8]:
subs_bs_voice_session = pd.read_csv(pathfiles + path_subs_bs_voice_session, sep=';', decimal=',')

In [9]:
subs_bs_voice_session['day'] = subs_bs_voice_session['START_TIME'].apply(lambda x:datetime.strptime(x, '%d.%m %H:%M:%S'))

In [11]:
subs_bs_voice_session.head(10)

,SK_ID,CELL_LAC_ID,VOICE_DUR_MIN,START_TIME,day
0,373,269094,0.001597,27.04 23:00:00,1900-04-27 23:00:00
1,373,269094,0.009583,29.04 17:00:00,1900-04-29 17:00:00
2,373,269094,0.003403,04.04 09:00:00,1900-04-04 09:00:00
3,373,269094,0.004653,08.04 17:00:00,1900-04-08 17:00:00
4,373,249478,0.002361,30.03 16:00:00,1900-03-30 16:00:00
5,373,249478,0.001389,30.04 21:00:00,1900-04-30 21:00:00
6,373,249478,0.005417,13.04 18:00:00,1900-04-13 18:00:00
7,373,249478,0.002986,05.05 11:00:00,1900-05-05 11:00:00
8,373,147305,0.007569,24.03 19:00:00,1900-03-24 19:00:00
9,373,182605,0.006458,26.03 08:00:00,1900-03-26 08:00:00


In [7]:
cells_2g = pd.read_csv(path + '2g.csv')
cells_3g = pd.read_csv(path + '3g.csv')
cells_4g = pd.read_csv(path + '4g.csv')

,Unnamed: 0,cell_id
0,0,262145
1,1,4
2,2,262150
3,3,262151
4,4,16


In [12]:
subs_bs_consumption = pd.read_csv(pathfiles + path_subs_bs_consumption, sep=';', decimal=',')

In [13]:
subs_bs_consumption.head()

,SK_ID,CELL_LAC_ID,MON,SUM_MINUTES,SUM_DATA_MB,SUM_DATA_MIN
0,1827,240277,01.03,0.000000,0.000002,0.000234
1,1827,355195,01.03,0.000072,0.000000,0.000000
2,1827,238554,01.04,0.000000,0.000019,0.000326
3,1827,350978,01.04,0.016576,0.002260,0.004329
4,1827,315010,01.04,0.012755,0.001186,0.002491


In [14]:
subs_bs_consumption['SPEED'] = subs_bs_consumption['SUM_DATA_MB'] / subs_bs_consumption['SUM_DATA_MIN']

In [18]:
subs_bs_consumption['SUM_DATA_MIN'].mean()

0.0029117627765275723

In [19]:
subs_bs_consumption['SPEED'].fillna(0).median()

0.0016005030152333591

In [24]:
subs_bs_consumption.merge(cells_2g, how='inner', left_on='CELL_LAC_ID', right_on='cell_id')['SPEED'].fillna(0).median()

0.0

In [25]:
subs_bs_consumption.merge(cells_3g, how='inner', left_on='CELL_LAC_ID', right_on='cell_id')['SPEED'].fillna(0).median()

0.0006215350460538761

In [26]:
subs_bs_consumption.merge(cells_4g, how='inner', left_on='CELL_LAC_ID', right_on='cell_id')['SPEED'].fillna(0).median()

0.009856861232536212

In [11]:
cells_2g.head()

,Unnamed: 0,cell_id
0,0,262145
1,1,4
2,2,262150
3,3,262151
4,4,16


In [8]:
columns_2g = ['T_DATE', 'CELL_LAC_ID', 'CELL_AVAILABILITY_2G', 'CSSR_2G', 'PSSR_2G', 'TBF_DROP_RATE_2G', 'TCH_DROP_RATE_2G']

In [14]:
columns_3g = ['T_DATE', 'CELL_LAC_ID', 'CELL_AVAILABILITY_3G', 'CSSR_3G', 'HSPDSCH_CODE_UTIL_3G', 'NODEB_CNBAP_LOAD_HARDWARE', 'PROC_LOAD_3G',
             'PSSR_3G', 'RAB_CS_BLOCKING_RATE_3G', 'RAB_CS_DROP_RATE_3G', 'RAB_PS_BLOCKING_RATE_3G', 'RAB_PS_DROP_RATE_3G',
             'RRC_BLOCKING_RATE_3G', 'RTWP_3G', 'SHO_FACTOR', 'TOTAL_DL_VOLUME_3G', 'UTIL_BRD_CPU_3G', 'UTIL_CE_DL_3G',
             'UTIL_CE_HW_DL_3G', 'UTIL_CE_UL_3G', 'UTIL_SUBUNITS_3G', 'TOTAL_UL_VOLUME_3G']

In [15]:
columns_4g = ['T_DATE', 'CELL_LAC_ID', 'CELL_AVAILABILITY_4G', 'DL_VOLUME_LTE', 'ERAB_PS_BLOCKING_RATE_LTE', 'ERAB_PS_BLOCKING_RATE_PLMN_LTE',
             'ERAB_PS_DROP_RATE_LTE', 'PART_CQI_QPSK_LTE', 'PART_MCS_QPSK_LTE', 'PSSR_LTE', 'RBU_AVAIL_DL', 'UL_VOLUME_LTE',
             'RBU_AVAIL_DL_LTE', 'RBU_AVAIL_UL', 'RBU_OTHER_DL', 'RBU_OTHER_UL', 'RBU_OWN_DL', 'RBU_OWN_UL', 'RRC_BLOCKING_RATE_LTE']

In [18]:
bs_avg_kpi = pd.read_csv(path + 'bs_avg_kpi.csv', sep=';', usecols=columns_4g, decimal=',')

In [19]:
bs_avg_kpi.merge(cells_4g, how='inner', left_on='CELL_LAC_ID', right_on='cell_id')[columns_4g].to_csv(path + 'bs_avg_kpi_4g.csv', index=False)